In [1]:
import requests
import re
from bs4 import BeautifulSoup

page =requests.get("https://www.imdb.com/list/ls091520106/")

soup2 =BeautifulSoup(page.content, 'html.parser')

In [2]:
films = soup2.find('div',{'class' : 'lister-list'})

f_links = []
for x in films.find_all('a', href=True):
    f_links.append(x['href'])

f_linksC = []
for i in range(1, len(f_links) - 1):
    if f_links[i] != '#void':
            if f_links[i] != f_links[i + 1]:
                if f_links[i][1] != 'n':
                    f_linksC.append(f_links[i])

filmingDates = []

loc = '/locations?ref_=tt_dt_dt'

for i in range(100):
    locUrl = 'https://www.imdb.com/' + f_linksC[i][1:-1] + loc
    locPage = requests.get(locUrl)
    locSoup = BeautifulSoup(locPage.content, 'html.parser')
    fd = locSoup.find('li', {'class':"ipl-zebra-list__item"})
    if fd:
        filmingDates.append(fd.text.strip())
    else:
        filmingDates.append('Not given.')

In [65]:
titles = []
descriptions = []
releases = []
directors = []
ratings = []
durations = []
genres = []
stars = []


for i in range(1, 201, 2):
    child = list(films.children)[i]
    filmString = child.text.replace('\n', "")
    #. \ + * ? [ ^ ] $ ( ) { } = !  | : -  
    flist = re.split("\||  ", filmString)
    fset = [x for x in flist if x != '']
    if 'Metascore' in fset:
        fset.remove('Metascore')
    if len(fset) > 1:
        t = fset[0]
        t = t[t.find('.') + 1:]
        rel = t[::-1][t[::-1].find(')') + 1:][:4][::-1]
        t = t[:t.find('(')]
        releases.append(int(rel))
        titles.append(t)
        durations.append(int(fset[1][:fset[1].find(' ')]))
        genres.append(fset[2])
            
        r = fset[3][:5]
        for l in r:
            if l.isalpha():
                r = r.replace(l,"")

        ratings.append(float(r))
        descriptions.append(fset[4])
        directors.append(fset[5][9:])
        s = fset[6][7:]
        s = s[::-1]
        s = s[s.find('V') + 1:]
        stars.append(s[::-1])

top_100 = {'title' : titles, 'description' : descriptions, 'release' : releases, 'director' :  directors, 'rating' : ratings, 'duration' : durations, 'genre' : genres, 'stars' : stars, 'filming dates' : filmingDates}

print(top_100)





{'title': ['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II', 'The Dark Knight', '12 Angry Men', "Schindler's List", 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Good, the Bad and the Ugly', 'Fight Club', 'Joker', 'The Lord of the Rings: The Fellowship of the Ring', 'Forrest Gump', 'Inception', 'Star Wars: Episode V - The Empire Strikes Back', 'The Lord of the Rings: The Two Towers', 'The Matrix', "One Flew Over the Cuckoo's Nest", 'Goodfellas', 'Seven Samurai', 'Se7en', 'City of God', 'Life Is Beautiful', 'The Silence of the Lambs', 'Star Wars: Episode IV - A New Hope', "It's a Wonderful Life", 'Saving Private Ryan', 'Spirited Away', 'The Green Mile', 'Léon: The Professional', 'Hara-Kiri', 'Interstellar', 'The Usual Suspects', 'The Lion King', 'American History X', 'Back to the Future', 'The Pianist', 'Modern Times', 'Terminator 2: Judgment Day', 'The Intouchables', 'Psycho', 'Gladiator', 'City Lights', 'The Departed', 'Whiplash', 'Once Upo

In [81]:
    #pandas stuff

#pandas dataframe with all
#normalized max_min (Rating/Duration (minutes))
#normalized0_1 (Rating / time to film)
import pandas as pd

#x normalized = (x – x minimum) / (x maximum – x minimum)

df = pd.DataFrame(top_100, index = list(range(1,101)))

rds = []
rts = []
for i in range(0, 100):
        rd = ratings[i] / durations[i]
        rds.append(rd)
df['Rating / Duration'] = rds